In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
pip install category_encoders 

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import category_encoders as ce

In [ ]:
data_train = pd.read_csv(r'/content/drive/MyDrive/Projects/Cost to Ship Sculpture/Data/train.csv')
data_test = pd.read_csv(r'/content/drive/MyDrive/Projects/Cost to Ship Sculpture/Data/test.csv')

data_train['Scheduled Date']  = pd.to_datetime(data_train['Scheduled Date'], format = '%m/%d/%y') 
data_train['Delivery Date']  = pd.to_datetime(data_train['Delivery Date'], format = '%m/%d/%y') 
data_train['date_diff'] = (data_train['Scheduled Date'].dt.day - data_train['Delivery Date'].dt.day)

data_test['Scheduled Date']  = pd.to_datetime(data_test['Scheduled Date'], format = '%m/%d/%y') 
data_test['Delivery Date']  = pd.to_datetime(data_test['Delivery Date'], format = '%m/%d/%y') 
data_test['date_diff'] = (data_test['Scheduled Date'].dt.day - data_test['Delivery Date'].dt.day)

for i in range(data_train.shape[0]) : 
        data_train.loc[i, 'Customer Location'] = data_train.loc[i, 'Customer Location'][-8:-6]
    
data_train.head(2)

In [ ]:
cols_to_drop = ['Customer Id', 'Artist Name','Artist Reputation','Scheduled Date','Delivery Date']
test_output = data_test.loc[:,'Customer Id'].tolist()
data_train = data_train.drop(cols_to_drop , axis  = 1)
data_test = data_test.drop(cols_to_drop , axis  = 1)


data_train['Cost'] = abs(data_train['Cost'])

In [ ]:
data_train

In [ ]:
from sklearn.impute import SimpleImputer

imputer_num = SimpleImputer(missing_values=np.nan, strategy = 'median')
data_train.iloc[:,[0, 1, 2]] = imputer_num.fit_transform(data_train.iloc[:,[0, 1, 2]])
data_test.iloc[:,[0, 1, 2]] = imputer_num.transform(data_test.iloc[:,[0, 1, 2]])

In [ ]:
imputer_cat = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
data_train.iloc[:,[3,9,12]] = imputer_cat.fit_transform(data_train.iloc[:,[3,9,12]])
data_test.iloc[:,[3,9,12]] = imputer_cat.transform(data_test.iloc[:,[3,9,12]])

In [ ]:
for i,j in enumerate(data_train.columns) : 
    print(i,' : ',j)

In [ ]:
from category_encoders import TargetEncoder
encoder = ce.TargetEncoder(cols= 'Customer Location') 

data_train.loc[:,'Customer Location'] = encoder.fit_transform(data_train['Customer Location'],data_train['Cost'])

data_test.loc[:,'Customer Location'] = encoder.transform(data_test.loc[:,'Customer Location'])

In [ ]:
X = data_train.loc[:, ['Height', 'Width', 'Weight', 'Material', 'Price Of Sculpture',
       'Base Shipping Price', 'International', 'Express Shipment',
       'Installation Included', 'Transport', 'Fragile', 'Customer Information',
       'Remote Location', 'Customer Location', 'date_diff']].values

y = data_train.loc[:,'Cost'].values

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size= 0.2)
X_test = data_test.values

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

cols_2_onehotencode = [3, 6, 7, 8, 9, 10, 11, 12]
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop = 'first'), cols_2_onehotencode)], remainder='passthrough')

X_train = ct.fit_transform(X_train)
X_val = ct.transform(X_val)
X_test = ct.transform(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
from datetime import datetime
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

start = datetime.now()

parameters = {
            'kernel' : ['poly', 'rbf'],
            'C': [0.1, 1, 100, 1000] ,
            'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
            'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]}


svr = SVR()

model = GridSearchCV(svr, cv = 10, param_grid=parameters, scoring = 'neg_mean_absolute_error', n_jobs = -1)
model.fit(X_train, y_train)


end = datetime.now()


print('Time taken : ', end -start)

In [ ]:
model.best_params_

In [ ]:
y_pred_test = model.predict(X_test)

test_out = pd.DataFrame({'Customer Id' : test_output, 'Cost' : y_pred_test})
print(test_out)

In [ ]:
test_out.to_csv('/content/drive/MyDrive/Projects/Cost to Ship Sculpture/Data/test_output2.csv', header = True, index = False)